### Проект

Имеется датасет, содержащий данные об известных личностях 20 и 21 века и статьях на Википедии, посвященных им.
Атрибуты:


sex - пол

birth_year - год рождения

latitude - широта города рождения

longitude - долгота города рождения

sphereas - сфера деятельности личности (политика, экономика, культура, духовная сфера)

article_languages - количество языков, на которые была переведена статья

page_views - количество просмотров статьи



Необходимо провести кластеризацию этих данных, чтобы разбить их на подгруппы
Кроме этого, на основе полученных данных провести анализ достаточности количества языков, на которые была переведена статья.

In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn import cluster
from sklearn.preprocessing import MinMaxScaler, normalize 
import math
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [5]:
def distance(i):
    if i >= len(normalized_data):
        return Z[i - len(normalized_data)][2]
    else:
        return sum(normalized_data.iloc[i]) / normalized_data.shape[1]
# рассчет расстояния между кластерами

In [6]:
def cluster_len(i):
    if i >= len(normalized_data):
        return Z[i - len(normalized_data)][3]
    else:
        return 1


In [7]:
def distance_split(c):
    kmeans = KMeansClusterer(2,cosine_distance)
    kmeans.fit(c)
    idk = np.argsort(kmeans.classification_probdist)
    idk0 = []
    idk1 = []
    k = 0
    for i in kmeans.classification_probdist[idk]:
        if i == 0:
            idk0.append(idk[k])
            k += 1
        else:
            idk1.append(idk[k])
            k += 1

    a0 = pd.DataFrame(c.values[idk0], c.index[idk0])
    a1 = pd.DataFrame(c.values[idk1], c.index[idk1])
    return a0, a1

#алгоритм разделения

In [8]:
def diana(c):
    if len(c) < 2:
        ind = np.argwhere(normalized_data.index == c.index[0])
        return ind[0][0]
    a0, a1 = distance_split(c)
    clusters.append(a0)
    clusters.append(a1)
    index1 = diana(a0)
    index2 = diana(a1)
    Z.append([
        index1,
        index2,
        distance(index1) + distance(index2),
        cluster_len(index1) + cluster_len(index2)
    ])

    return len(Z) - 1 + len(normalized_data)


# реализация алгоритма Diana

In [9]:
Z

[]

In [10]:
data

,sex_Male,sex_Female,birth_year,latitude,longitude,article_languages,page_views,sphereas
0,1,0,1928,-32.95000,-60.66667,129,54896869,2
1,1,0,1929,33.75500,-84.39000,116,28854653,2
2,1,0,1935,34.25972,-88.72583,118,55896331,1
3,1,0,1904,42.26667,2.96500,122,37482606,1
4,0,1,1926,34.05000,-118.25000,143,63671491,1
...,...,...,...,...,...,...,...,...
279,0,1,1990,38.25000,-85.76667,42,35058386,1
280,1,0,1990,-37.81361,144.96306,29,21614952,1
281,1,0,1987,48.23306,-101.29222,32,23292706,1
282,0,1,1981,42.99944,-82.30889,27,21407381,2


In [11]:
data=pd.read_excel("СпецГлМат_ДЗ1_ФунтовСС.xlsx",sheet_name="Выборка для кластеризации")

Z = []
clusters = []

scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data), columns = data.columns)

normalized_data = pd.DataFrame(normalize(scaled_data), columns = data.columns)



index = diana(normalized_data)

fig = plt.figure(figsize=(25, 10), dpi=100)
plt.title('Divisive algorithm')
dn = dendrogram(Z, labels=data.index, orientation='top', color_threshold=1500)
plt.show()


# Оптимальное количество кластеров - 3

NameError: name 'cosine_distance' is not defined

In [4]:
data=pd.read_excel("СпецГлМат_ДЗ1_ФунтовСС.xlsx",sheet_name="Выборка для регрессии",index_col= False)
scaled_data = pd.DataFrame(scaler.fit_transform(data), columns = data.columns)

normalized_data = pd.DataFrame(normalize(scaled_data), columns = data.columns)
#linear_regression_model = SGDRegressor(tol=0.0001, eta0=0.01)
linear_regression_model = LinearRegression()
X = scaled_data[["sex_Male","sex_Female","birth_year","latitude","longitude","sphereas_Spiritual","sphereas_Social","sphereas_Economic","page_views"]]
y = data['article_languages']

In [12]:
corr = normalized_data.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

NameError: name 'sns' is not defined

In [5]:
linear_regression_model.fit(X, y)
predictions = linear_regression_model.predict(X)
mse = mean_squared_error(data['article_languages'], predictions)
R2 = r2_score(y, predictions)
R2_adj = 1 - (1- R2) * ( len(y) - 1 ) / ( len(y) - X.shape[1] - 1 )

In [8]:
x = sm.add_constant(X)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      article_languages   R-squared:                       0.587
Model:                            OLS   Adj. R-squared:                  0.575
Method:                 Least Squares   F-statistic:                     48.78
Date:                Thu, 18 May 2023   Prob (F-statistic):           1.65e-48
Time:                        14:12:21   Log-Likelihood:                -1199.1
No. Observations:                 284   AIC:                             2416.
Df Residuals:                     275   BIC:                             2449.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 70.8599      4

In [83]:
mse

272.14731528079136

In [80]:
R2

0.5866084783959782

In [81]:
R2_adj

0.573029924766649

In [82]:
clusters[0]

,0,1,2,3,4,5,6,7
0,0.763582,0.0,0.212543,0.037987,0.240051,0.450205,0.212677,0.254527
148,0.654012,0.0,0.532649,0.052050,0.225576,0.189021,0.095536,0.436008
149,0.544735,0.0,0.471730,0.392851,0.248321,0.188926,0.311710,0.363157
150,0.690822,0.0,0.498531,0.496979,0.143351,0.079864,0.017746,0.000000
151,0.611740,0.0,0.428849,0.586064,0.303653,0.074257,0.025924,0.000000
...,...,...,...,...,...,...,...,...
9,0.736217,0.0,0.333954,0.422552,0.191873,0.272358,0.238569,0.000000
10,0.713319,0.0,0.286798,0.551666,0.081735,0.235024,0.206567,0.000000
75,0.703964,0.0,0.362868,0.573538,0.159761,0.134282,0.016137,0.000000
79,0.687786,0.0,0.319076,0.578922,0.188301,0.043732,0.007682,0.229262


In [85]:
clusters[0].index

Int64Index([  0, 148, 149, 150, 151, 152, 155, 158, 159, 160,
            ...
             70,  11,  77,  72,  74,   9,  10,  75,  79,  73],
           dtype='int64', length=191)

In [86]:
cluster0 = data.iloc[clusters[0].index]

In [87]:
cluster0.describe() 

,sex_Male,sex_Female,birth_year,latitude,longitude,article_languages,page_views,sphereas
count,191.0,191.0,191.000000,191.000000,191.000000,191.000000,1.910000e+02,191.000000
mean,1.0,0.0,1959.356021,35.124500,-53.289954,70.759162,3.467130e+07,1.581152
std,0.0,0.0,20.662423,19.939689,62.805090,27.006912,1.793183e+07,0.889980
min,1.0,0.0,1901.000000,-37.813610,-157.816670,29.000000,2.001568e+07,1.000000
25%,1.0,0.0,1944.000000,34.040000,-89.971070,52.500000,2.412118e+07,1.000000
50%,1.0,0.0,1963.000000,40.688890,-74.005900,67.000000,2.924053e+07,1.000000
75%,1.0,0.0,1976.000000,43.171670,-2.931285,84.000000,3.817964e+07,2.000000
max,1.0,0.0,1998.000000,59.950000,151.209440,200.000000,1.452506e+08,4.000000


In [88]:
cluster2 = data.iloc[clusters[1].index]

In [89]:
cluster2.describe() # в кластер попали известные женщины

,sex_Male,sex_Female,birth_year,latitude,longitude,article_languages,page_views,sphereas
count,93.0,93.0,93.000000,93.000000,93.000000,93.000000,9.300000e+01,93.000000
mean,0.0,1.0,1973.494624,37.072609,-71.872632,66.408602,3.568225e+07,1.107527
std,0.0,0.0,18.890775,11.047262,50.262563,22.650577,1.757670e+07,0.311461
min,0.0,1.0,1907.000000,-26.188330,-157.816670,27.000000,2.024140e+07,1.000000
25%,0.0,1.0,1967.000000,33.755000,-115.566670,48.000000,2.319981e+07,1.000000
50%,0.0,1.0,1980.000000,36.677780,-83.166940,66.000000,2.921449e+07,1.000000
75%,0.0,1.0,1986.000000,42.480560,-73.352220,78.000000,4.025286e+07,1.000000
max,0.0,1.0,1994.000000,52.918000,74.880000,143.000000,1.045626e+08,2.000000


In [90]:
cluster3 = data.iloc[clusters[2].index]

In [91]:
cluster3.describe() # в кластер попали деятели политики, экономики и социальной сферы

,sex_Male,sex_Female,birth_year,latitude,longitude,article_languages,page_views,sphereas
count,126.0,126.0,126.000000,126.000000,126.000000,126.000000,1.260000e+02,126.0
mean,1.0,0.0,1961.293651,38.161699,-56.261194,64.063492,3.473185e+07,1.0
std,0.0,0.0,17.928332,15.959742,66.040793,20.752540,1.812165e+07,0.0
min,1.0,0.0,1904.000000,-37.813610,-157.816670,29.000000,2.004257e+07,1.0
25%,1.0,0.0,1947.000000,34.178195,-95.246463,51.250000,2.465000e+07,1.0
50%,1.0,0.0,1964.000000,40.692780,-76.711270,63.000000,2.937527e+07,1.0
75%,1.0,0.0,1974.000000,46.115000,-3.000000,75.000000,3.799896e+07,1.0
max,1.0,0.0,1998.000000,55.943610,151.209440,147.000000,1.452506e+08,1.0
